# Fama-French Industries and Short-Sales Constraints

## Frontiers with and without shorting

In [2]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar

SolverOptions["show_progress"] = False


In [3]:
# Read data and clean-up missing data (coded -99.99)
ff48 = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]

# Clean-up missings
for c in ff48.columns:
    ff48[c] = np.where(ff48[c]==-99.99, np.nan, ff48[c])
ff48 = ff48/100

In [4]:
# Estimate inputs from historical data
mns = ff48.mean()
sds = ff48.std()
C = ff48.corr()
cov = np.diag(sds) @ C @ np.diag(sds)


In [5]:
# Define portfolio class
class portfolio:

    def __init__(self, means, cov, Shorts):
        self.means = np.array(means)
        self.cov = np.array(cov)
        self.Shorts = Shorts
        self.n = len(means)
        if Shorts:
            w = np.linalg.solve(cov, np.ones(self.n))
            self.GMV = w / np.sum(w)
            w = np.linalg.solve(cov, means)
            self.piMu = w / np.sum(w)
        else:
            n = self.n
            Q = matrix(cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.ones(n), (1, n), tc="d")
            b = matrix([1], (1, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            self.GMV = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def frontier(self, m):
        if self.Shorts:
            gmv = self.GMV
            piMu = self.piMu
            m1 = gmv @ self.means
            m2 = piMu @ self.means
            a = (m - m2) / (m1 - m2)
            return a * gmv + (1 - a) * piMu
        else:
            n = self.n
            Q = matrix(self.cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.vstack((np.ones(n), self.means)), (2, n), tc="d")
            b = matrix([1, m], (2, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def tangency(self, r):
        if self.Shorts:
            w = np.linalg.solve(self.cov, self.means - r)
            return w / np.sum(w)
        else:
            def f(m):
                w = self.frontier(m)
                mn = w @ self.means
                sd = np.sqrt(w.T @ self.cov @ w)
                return - (mn - r) / sd
            m = minimize_scalar(f, bounds=[max(r, np.min(self.means)), max(r, np.max(self.means))], method="bounded").x
            return self.frontier(m)

    def optimal(self, raver, rs=None, rb=None):
        n = self.n
        if self.Shorts:
            if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n + 2, 1), tc="d")
                G = np.zeros((2, n + 2))
                G[0, 0] = G[1, 1] = -1
                G = matrix(G, (2, n+2), tc="d")
                h = matrix([0, 0], (2, 1), tc="d")
                A = matrix([1, -1] + n*[1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
            else:
                w = np.linalg.solve(self.cov, self.means)
                a = np.sum(w)
                return (a/raver)*self.piMu + (1-a/raver)*self.GMV
        else:
           if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n+2, 1), tc="d")
                G = matrix(-np.identity(n + 2), tc="d")
                h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
                A = matrix([1, -1] + n * [1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
           else:
                Q = matrix(raver * self.cov, tc="d")
                p = matrix(-self.means, (n, 1), tc="d")
                G = matrix(-np.identity(n), tc="d")
                h = matrix(np.zeros(n), (n, 1), tc="d")
                A = matrix(np.ones(n), (1, n), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else None


In [6]:
# Frontier with shorting              
ss  = portfolio(mns.values,cov.values, Shorts = True)
frontier_ss  = [ss.frontier(m) for m in np.linspace(mns.min(), mns.max(),50)]
mu_ss = [wgt @ mns for wgt in frontier_ss]
sd_ss = [np.sqrt(wgt @ cov @ wgt) for wgt in frontier_ss]

# Frontier without shorting
noss= portfolio(mns.values,cov.values, Shorts = False)
frontier_noss= [noss.frontier(m) for m in np.linspace(mns.min(), mns.max(),50)]
mu_noss = [wgt @ mns for wgt in frontier_noss]
sd_noss = [np.sqrt(wgt @ cov @ wgt) for wgt in frontier_noss]

In [7]:
# Plot the frontiers
import plotly.graph_objects as go
trace_ss  = go.Scatter(x=sd_ss, y=mu_ss, mode="lines", name='With shorting')
trace_noss= go.Scatter(x=sd_noss, y=mu_noss, mode="lines", name='Without shorting')
fig = go.Figure()
fig.add_trace(trace_ss)
fig.add_trace(trace_noss)
fig.show()

## Performance out-of-sample

In [120]:
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
df = ff48.join(ff3['RF'])
df = df.loc['1970-01':].copy()  # There is missing data prior to 1970

# Estimation window
WINDOW = 120
# Risk aversion 
RAVER = 3


In [ ]:

asset_list = ff48.columns
dates = df.index[WINDOW:]
df['retp_ss'] = np.nan
df['retp_noss'] = np.nan
df['retp_ew'] = np.nan
df['sum_risky_ss'] = np.nan
df['sum_risky_noss'] = np.nan
df['sum_risky_ew'] = np.nan

# DataFrame for tracking industry weights
df_wgts = pd.DataFrame(dtype=float, columns = pd.MultiIndex.from_product([['ss','noss','ew'], asset_list]), index=dates)


for d in dates:
    data= df.loc[:d-1,asset_list] 
    mns = data.mean()
    sds = data.std()
    C   = data.corr()
    cov = np.diag(sds)@ C @ np.diag(sds)
    n = len(mns)
    rf = df.loc[d,'RF']
    # print(f'Date: {d}\n')

    # Optimal portfolios w/ shorts
    ss  = portfolio(mns.values,cov.values, Shorts = True)
    wgt_ss = ss.optimal(RAVER, rf, rf)
    if wgt_ss.sum()<0:
        wgt_ss = 0.0*wgt_ss
    df.loc[d,'sum_risky_ss'] = wgt_ss.sum()
    df.loc[d,'retp_ss'] = rf + wgt_ss @ (df.loc[d,asset_list] - rf)
    df_wgts.loc[d,('ss',slice(None))] = wgt_ss
    # print(f'\n\tWith Short-sales:')
    # print(f'\n\tTotal Risky Weight: {wgt_ss.sum(): .2f}\n\tMin Weight: {wgt_ss.min(): .2f}\n\tMax Weight: {wgt_ss.max(): .2f}')

    # Optimal portfolios w/o shorts
    noss  = portfolio(mns.values,cov.values, Shorts = False)
    wgt_noss = noss.optimal(RAVER, rf, rf)
    if wgt_noss.sum()<0:
        wgt_noss = 0.0*wgt_noss    
    df.loc[d,'sum_risky_noss'] = wgt_noss.sum()        
    df.loc[d,'retp_noss'] = rf + wgt_noss @ (df.loc[d,asset_list] - rf)
    df_wgts.loc[d,('noss',slice(None))] = wgt_noss
    # print(f'\n\tWithout Short-sales:')
    # print(f'\n\tTotal Risky Weight: {wgt_noss.sum(): .2f}\n\tMin Weight: {wgt_noss.min(): .2f}\n\tMax Weight: {wgt_noss.max(): .2f}')    

    # Equal-weighted
    mns = mns.mean()*np.ones(n)
    sds = sds.mean()*np.ones(n)
    cov = np.diag(sds) @ np.identity(n) @ np.diag(sds)
    ew = portfolio(mns, cov, Shorts=True)
    wgt_ew = ew.optimal(RAVER,rf,rf)
    if wgt_ew.sum()<0:
        wgt_ew = 0.0*wgt_ew    
    df.loc[d,'retp_ew'] = rf + wgt_ew @ (df.loc[d,asset_list] - rf)
    df.loc[d,'sum_risky_ew'] = wgt_ew.sum() 
    df_wgts.loc[d,('ew',slice(None))] = wgt_ew       
    # print(f'\n\tEqual-weighted:')
    # print(f'\n\tTotal Risky Weight: {wgt_ew.sum(): .2f}\n\tMin Weight: {wgt_ew.min(): .2f}\n\tMax Weight: {wgt_ew.max(): .2f}')        


In [122]:
portrets = df[['retp_ss','retp_noss','retp_ew','RF']+['sum_risky_' + v for v in ['ss','noss','ew']]]
portrets = portrets[portrets.retp_ss.isnull()==False]
portrets['Date'] = portrets.index.to_timestamp('M')
portrets.describe()

,retp_ss,retp_noss,retp_ew,RF,sum_risky_ss,sum_risky_noss,sum_risky_ew
count,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000
mean,0.030860,0.015912,0.201497,0.003257,2.253537,1.948056,25.885522
std,0.221811,0.080474,1.325706,0.002932,0.836792,0.588367,10.716143
min,-0.928871,-0.353533,-7.324065,0.000000,0.000000,0.148932,0.000000
25%,-0.068482,-0.027020,-0.316690,0.000600,1.784108,1.683072,21.320344
50%,0.037419,0.018750,0.178895,0.003100,2.491607,2.078718,28.934321
75%,0.148590,0.064718,0.934763,0.004800,2.929529,2.415695,34.571798
max,0.847550,0.285406,5.676907,0.013500,3.301256,2.652768,38.367298


In [142]:
# Realized sharpe ratios for each strategy
def sr(varname):
    avg_exret = portrets[varname].mean() - portrets['RF'].mean()
    sd = portrets[varname].std()
    return avg_exret / sd

sr_ss = sr('retp_ss')
print(sr_ss)

sr_noss = sr('retp_noss')
print(sr_noss)

sr_ew = sr('retp_ew')
print(sr_ew)

0.12444217073245464
0.15725888165143143
0.1495352509876081


In [152]:
# Plot the time-series of returns (sum of risky portfolio weights as hoverdata)
fig = go.Figure()

label_dict = {'ss': 'With short sales',
            'noss': 'Without short sales'}        

for model in ['ss', 'noss']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Date: %{x}<br>"
    string += "Return: %{y:0.1%}<br>"
    string += "Total Weight in Risky Portfolio: %{customdata: 0.2%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=portrets['Date'], y=portrets['retp_'+model], customdata=portrets['sum_risky_'+model], 
        hovertemplate=string, name = label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Return"
fig.layout.xaxis["title"] = "Date"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [154]:
# Plot the time-series of sum of risky portfolio weights (returns as hoverdata)
fig = go.Figure()

label_dict = {'ss': 'With short sales',
            'noss': 'Without short sales'}        

for model in ['ss', 'noss']:
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Date: %{x}<br>"
    string += "Total Weight in Risky Portfolio: %{y:0.1%}<br>"
    string += "Return %{customdata: 0.2%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=portrets['Date'], y=portrets['sum_risky_'+model], customdata=portrets['retp_'+model], 
        hovertemplate=string, name = label_dict[model])
    fig.add_trace(trace)
fig.layout.yaxis["title"] = "Total Risky Weight"
fig.layout.xaxis["title"] = "Date"
fig.update_yaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [170]:
# Plot the time-series of cumulative returns (sum of risky portfolio weights as hoverdata)
label_dict = {'ss': 'With short sales',
            'noss': 'Without short sales'}        

for model in ['ss', 'noss']:
    fig = go.Figure()
    string =  "Strategy: " + label_dict[model] +" <br>"
    string += "Date: %{x}<br>"
    string += "Cumulative Return: $%{y:0.2f}<br>"
    string += "Total Weight in Risky Portfolio: %{customdata: 0.2%} <br>"
    string += "<extra></extra>"

    wgt_list = ['wgt_cal_'+ model] + ['wgt'+asset + "_" + model for asset in asset_list]
    trace=go.Scatter(x=portrets['Date'], y=(1+portrets['retp_'+model]).cumprod(), customdata=portrets['sum_risky_'+model], 
        hovertemplate=string, name = label_dict[model])
    fig.add_trace(trace)
    fig.layout.yaxis["title"] = "FV of $1"
    fig.layout.xaxis["title"] = "Date"
    fig.update_yaxes(tickformat=".2f")
    fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01),
         title=label_dict[model])
    fig.show()